In [1]:
import sys
import os
import requests
import json

In [2]:
url = "https://desc-tom.lbl.gov"
# url = "https://desc-tom-rknop-dev.lbl.gov"

# Somehow you need to set the "username" and "password" variables.
# Do NOT hardcode the password into any code that will be world
# readable.  You can use the python input statement, or, as I do
# here, read the password from a file that is only readable by
# you.  (This isn't perfect, but it's better than having the
# password sit around in a world readable file.)
username = "rknop"
with open( os.path.join( os.getenv("HOME"), "secrets", "tom_rknop_passwd" ) ) as ifp:
    password = ifp.readline().strip()

In [11]:
# This mess of code is just to log in to the Tom.  There is some
# dancing around necessary to get the "csrftoken" cookie that Django
# demands for queries you send in.
#
# The detection of failed login is not very good; it's scraping
# the text that an interactive failed login would have received.
# Sometime I should figure out how to create a API version of
# a login.
#
# Once this is done, the rqs object is the thing you use for further
# communcation.

rqs = requests.session()
rqs.get( f'{url}/accounts/login/' )
res = rqs.post( f'{url}/accounts/login/',
                data={ "username": username,
                       "password": password,
                       "csrfmiddlewaretoken": rqs.cookies['csrftoken'] } )
if res.status_code != 200:
    raise RuntimeError( f"Failed to log in; http status: {res.status_code}" )
if 'Please enter a correct' in res.text:
    raise RuntimeError( "Failed to log in.  I think.  Put in a debug break and look at res.text" )
rqs.headers.update( { 'X-CSRFToken': rqs.cookies['csrftoken'] } )

In [8]:
# TO send a query, just pass the query as a string, and any
# substitutions as a dict, to the post method of your rqs object.
# The query and the substitutions will be fed directly
# to psycopg2.  (The database user that the web API is using
# is a read-only user, so you shouldn't be able to do damage
# other than DOS-type things.)

query = 'SELECT * FROM elasticc_diasource WHERE "diaObjectId"=%(id)s ORDER BY "midPointTai"'
subdict = { "id": 10208128  }
result = rqs.post( f'{url}/db/runsqlquery/',
                   json={ 'query': query, 'subdict': subdict } )

In [10]:
# If all is well, in result.txt there is json.  It will have a field
# 'status' that is 'ok' if all is well, or 'error' if all is not.
# If there was an error, there will be a field 'error' with the 
# error message; you can use this to debug your SQL.  If the
# query succeeded, there will be a field 'rows' that is an
# array of dicts with the results of your query.

if result.status_code != 200:
        sys.stderr.write( f"ERROR: got status code {result.status_code} ({result.reason})\n" )
else:
    data = json.loads( result.text )
    if ( 'status' not in data ) or ( data['status'] != 'ok' ):
        if ( 'error' in data ):
            sys.stderr.write( f"Error return: {data['error']}" )
        else:
            sys.stderr.write( "Got unexpected response\n" )
    else:
        for row in data['rows']:
            print( f'Object: {row["diaObjectId"]:<20d}, Source: {row["diaSourceId"]:<20d}, '
                   f'MJD: {row["midPointTai"]:9.2f}, Flux: {row["psFlux"]:10.2f}' )

Object: 10208128            , Source: 20416256034         , MJD:  60379.30, Flux:    5535.07
Object: 10208128            , Source: 20416256035         , MJD:  60379.33, Flux:    3239.56
Object: 10208128            , Source: 20416256036         , MJD:  60379.35, Flux:    1773.24
Object: 10208128            , Source: 20416256037         , MJD:  60381.34, Flux:    4766.27
Object: 10208128            , Source: 20416256038         , MJD:  60381.37, Flux:    3055.83
Object: 10208128            , Source: 20416256039         , MJD:  60390.21, Flux:    3879.07
Object: 10208128            , Source: 20416256042         , MJD:  60398.33, Flux:    2266.00
Object: 10208128            , Source: 20416256044         , MJD:  60399.20, Flux:    3771.70
Object: 10208128            , Source: 20416256045         , MJD:  60399.22, Flux:    3678.21
Object: 10208128            , Source: 20416256046         , MJD:  60401.23, Flux:    3876.19
Object: 10208128            , Source: 20416256047         , MJD:  6040